In [ ]:
import os
import keras
from keras.datasets import cifar10
from keras.layers import Input, Conv2D, Dense, MaxPooling2D, Flatten, Activation, BatchNormalization, Dropout
from keras.models import Model
from keras import optimizers
from keras.callbacks import LearningRateScheduler, TensorBoard
import matplotlib.pyplot as plt


#Note we have tried different parameters for different combinations however for just one code we have provided this condensed code


num_classes = 10
batch_size = 64
epochs = 25
iterations = 782
DATA_FORMAT = 'channels_last'

# Define the directory for TensorBoard logs
log_dir = './logs'
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

log_filepath = os.path.join(log_dir, 'tensorboard_logs')

def color_preprocessing(x_train, x_test):
    x_train = x_train.astype('float32') / 255
    x_test = x_test.astype('float32') / 255
    return x_train, x_test

def scheduler(epoch):
    if epoch < 100:
        return 0.01
    # if epoch < 200:
    #     return 0.001
    return 0.0001

# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
x_train, x_test = color_preprocessing(x_train, x_test)

def medium_deep_net(img_input, activation, dropout_rate=0.5, optimizer=None):
    x = Conv2D(32, (3, 3), padding='same', activation=activation)(img_input)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2))(x)
    x = Dropout(dropout_rate)(x)  # Dropout after Conv2D

    x = Conv2D(64, (3, 3), padding='same', activation=activation)(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2))(x)
    x = Dropout(dropout_rate)(x)  # Dropout after Conv2D

    x = Flatten()(x)

    x = Dense(256, activation=activation)(x)
    x = BatchNormalization()(x)
    x = Dropout(dropout_rate)(x)  # Dropout after Dense
    x = Dense(128, activation=activation)(x)
    x = BatchNormalization()(x)
    x = Dropout(dropout_rate)(x)  # Dropout after Dense

    output = Dense(num_classes, activation='softmax')(x)

    model = Model(img_input, output)
    if optimizer:
        model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

def train_model(model, epochs=epochs):
    tb_cb = TensorBoard(log_dir=log_filepath, histogram_freq=0)
    change_lr = LearningRateScheduler(scheduler)
    cbks = [change_lr, tb_cb]
    model.summary()
    history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, callbacks=cbks, validation_data=(x_test, y_test))
    return history

activation_functions = ['relu', 'tanh', 'sigmoid', 'gelu']
optimizers_list = [
    {'name': 'SGD with Momentum', 'optimizer': optimizers.SGD(lr=0.01, momentum=0.9, nesterov=True)},
    {'name': 'SGD without Momentum', 'optimizer': optimizers.SGD(lr=0.01)},
    {'name': 'RMSprop', 'optimizer': optimizers.RMSprop()},
    {'name': 'Adam', 'optimizer': optimizers.Adam()},
]
dropout_rates = [0.25, 0.5, 0.75]  # Define different dropout rates to try

for activation in activation_functions:
    for dropout_rate in dropout_rates:
        for opt in optimizers_list:
            print(f"Training model with activation: {activation}, dropout rate: {dropout_rate}, optimizer: {opt['name']}")
            model = medium_deep_net(Input(shape=(32, 32, 3)), activation=activation, dropout_rate=dropout_rate, optimizer=opt['optimizer'])
            history = train_model(model)

            # Evaluate the model on test data
            test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
            print(f'Test accuracy with {activation} activation, dropout rate {dropout_rate}, and {opt["name"]} optimizer: {test_acc}')

print('Experimentation completed!')
